# 03 Imports
> standard imports

In [1]:
# default_exp imports

In [2]:
# export

__all__ = ["pd","np","partial","Path","json","Counter","list_vc","col_list_vc",
           "plt","os","sys","glob","Image",]
import pandas as pd

def list_vc(
    df, colname: str, value: str
) -> pd.DataFrame:
    """
    count the values in a column
        that each cell is a list
    """
    return df[colname].list_vc(value)

def col_list_vc(
    col, value: str
) -> pd.DataFrame:
    """
    count the values in a column
        that each cell is a list
    """
    return pd.DataFrame(
        col.apply(lambda x: value in x).value_counts()
    )

pd.DataFrame.vc = lambda self,col:pd.DataFrame(self[col].value_counts())
pd.Series.list_vc = col_list_vc
pd.DataFrame.list_vc = list_vc

import numpy as np
from pathlib import Path
import json
from functools import partial
from collections import Counter


import os
import sys

try:
    from matplotlib import pyplot as plt
except:
    pass

try:
    from forgebox.widgets import search_box,paginate
    pd.DataFrame.search_box = search_box
    pd.DataFrame.paginate = paginate
except:
    pass

try:
    import torch
    from torch import nn
    from torch.utils.data.dataset import Dataset
    from torch.utils.data.dataloader import DataLoader
    __all__+=["nn","torch","Dataset","DataLoader"]
except:
    pass

try:
    from ipywidgets import interact, interact_manual
    from IPython.display import HTML
    __all__+=["interact","interact_manual","HTML"]
except:
    pass

try:
    import redis
    def read_redis(key, host='localhost', port=6379, db=3):
        rd = redis.Redis(host=host, port=port, db=db)
        return pd.read_json(rd.get(key))
    
    __all__+=["read_redis",]
    
    def to_redis(self, key:str, con:redis.Redis = None, ex:int=600):
        """
        Save dataframe to redis
        """
        if con is None:
            con = redis.Redis(host='localhost', port=6379, db=3)
        return con.set(key, self.to_json())
        
    pd.DataFrame.to_redis = to_redis
    
    def df_cache(ex=600, host='localhost', port=6379, db=3):
        """
        Caching pd.DataFrame, use as decorator
        
        @df_cache(ex=300)
        def get_table_abc(uuid):
            return pd.read_sql(f"select * from abc where uuid='{uuid}'", con=conn)
            
        within 5 minutes, under same uuid, the code will query data base only once
        """
        rd = redis.Redis(host=host, port=port, db=db)
        def decorator(f):
            def wrapper(*args, **kwargs):
                key = "_".join(str(i).lower() for i in args)
                if len(kwargs)>0:
                    key += "_"
                    key += "_".join(f"{k}-{v}" for k,v in kwargs.items())
                if rd.exists(key):
                    return pd.read_json(rd.get(key))
                df = f(*args, **kwargs)
                df.to_redis(key, con=rd)
                return df

            return wrapper
        return decorator
    __all__+=["df_cache",]
        
except:
    pass

from glob import glob
from PIL import Image

Path.ls = lambda x:os.listdir(x)